In [2]:


# Import dependencies
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
# from sklearn.preprocessing import LabelEncoder


In [3]:
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect("fraud_detection.db")

# Read the data from the database
X_train = pd.read_sql_query("SELECT * FROM X_train", conn)
y_train = pd.read_sql_query("SELECT * FROM y_train", conn)["isFraud"]
X_test = pd.read_sql_query("SELECT * FROM X_test", conn)
y_test = pd.read_sql_query("SELECT * FROM y_test", conn)["isFraud"]

# Close the connection
conn.close()

In [4]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [5]:
#  Define the parameter grids for  XGBoost

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [6, 10, 15],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.5, 1],
    'colsample_bytree': [0.5, 1]
}

In [ ]:
# Perform GridSearchCV for XGBoost
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_grid = GridSearchCV(xgb, xgb_param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
xgb_grid.fit(X_train, y_train)

In [ ]:
print("Best XGBoost hyperparameters: ", xgb_grid.best_params_)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# XGBoost model with the best hyperparameters
best_xgb = xgb_grid.best_estimator_
xgb_predictions = best_xgb.predict(X_test)
xgb_probs = best_xgb.predict_proba(X_test)[:, 1]

In [ ]:
print("XGBoost Classification Report:")
print(classification_report(y_test, xgb_predictions))
print("ROC AUC Score: ", roc_auc_score(y_test, xgb_probs))